In [33]:
import requests
import pandas as pd
import datetime as dt
from datetime import timedelta
import time
import json

print("Libraries imported successfully!")


Libraries imported successfully!


In [ ]:
# Binance API configuration
BASE_URL = "https://api.binance.com/api/v3/klines"
SYMBOL = "BTCUSDC"
INTERVAL = "1h"  # 1 hour timeframe

# Calculate date range (1 year from last month)
end_time = (dt.datetime.now().replace(day=1) - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
start_time = end_time - timedelta(days=365)

print(f"Fetching Bitcoin price data from {start_time.strftime('%Y-%m-%d %H:%M:%S')} to {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Symbol: {SYMBOL}")
print(f"Interval: {INTERVAL}")


Fetching Bitcoin price data from 2024-06-30 00:00:00 to 2025-06-30 00:00:00
Symbol: ARBUSDC
Interval: 1h


In [35]:
def fetch_binance_klines(symbol, interval, start_time, end_time, limit=1000):
    """
    Fetch kline data from Binance API
    
    Parameters:
    - symbol: Trading pair symbol (e.g., 'BTCUSDT')
    - interval: Kline interval (e.g., '1h', '1d')
    - start_time: Start time as timestamp (milliseconds)
    - end_time: End time as timestamp (milliseconds)
    - limit: Number of klines to return (max 1000)
    
    Returns:
    - List of kline data
    """
    
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': int(start_time),
        'endTime': int(end_time),
        'limit': limit
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def datetime_to_timestamp(dt_obj):
    """Convert datetime object to millisecond timestamp"""
    return int(dt_obj.timestamp() * 1000)

print("Functions defined successfully!")


Functions defined successfully!


In [36]:
def fetch_all_bitcoin_data(symbol, interval, start_time, end_time):
    """
    Fetch all Bitcoin data for the specified time range
    Handles pagination automatically
    """
    
    all_data = []
    current_start = start_time
    
    # Convert datetime to timestamps
    start_timestamp = datetime_to_timestamp(current_start)
    end_timestamp = datetime_to_timestamp(end_time)
    
    total_hours = int((end_timestamp - start_timestamp) / (1000 * 60 * 60))
    print(f"Total hours to fetch: {total_hours}")
    
    batch_count = 0
    
    while current_start < end_time:
        # Calculate end time for this batch (1000 hours or until end_time)
        batch_end = current_start + timedelta(hours=999)  # 999 to be safe
        if batch_end > end_time:
            batch_end = end_time
            
        start_ts = datetime_to_timestamp(current_start)
        end_ts = datetime_to_timestamp(batch_end)
        
        print(f"Fetching batch {batch_count + 1}: {current_start.strftime('%Y-%m-%d %H:%M')} to {batch_end.strftime('%Y-%m-%d %H:%M')}")
        
        # Fetch data for this batch
        batch_data = fetch_binance_klines(symbol, interval, start_ts, end_ts)
        
        if batch_data:
            all_data.extend(batch_data)
            print(f"  → Fetched {len(batch_data)} records")
        else:
            print("  → Failed to fetch data for this batch")
            break
            
        # Move to next batch
        current_start = batch_end + timedelta(hours=1)
        batch_count += 1
        
        # Add a small delay to avoid hitting rate limits
        time.sleep(0.1)
    
    print(f"Total records fetched: {len(all_data)}")
    return all_data

print("Data fetching function ready!")


Data fetching function ready!


In [37]:
# Fetch all Bitcoin data
print("Starting data fetch...")
print("=" * 50)

bitcoin_data = fetch_all_bitcoin_data(SYMBOL, INTERVAL, start_time, end_time)

if bitcoin_data:
    print("=" * 50)
    print("Data fetch completed successfully!")
else:
    print("Failed to fetch data. Please check your internet connection and try again.")


Starting data fetch...
Total hours to fetch: 8760
Fetching batch 1: 2024-06-30 00:00 to 2024-08-10 15:00
  → Fetched 1000 records
Fetching batch 2: 2024-08-10 16:00 to 2024-09-21 07:00
  → Fetched 1000 records
Fetching batch 3: 2024-09-21 08:00 to 2024-11-01 23:00
  → Fetched 1000 records
Fetching batch 4: 2024-11-02 00:00 to 2024-12-13 15:00
  → Fetched 1000 records
Fetching batch 5: 2024-12-13 16:00 to 2025-01-24 07:00
  → Fetched 1000 records
Fetching batch 6: 2025-01-24 08:00 to 2025-03-06 23:00
  → Fetched 1000 records
Fetching batch 7: 2025-03-07 00:00 to 2025-04-17 15:00
  → Fetched 1000 records
Fetching batch 8: 2025-04-17 16:00 to 2025-05-29 07:00
  → Fetched 1000 records
Fetching batch 9: 2025-05-29 08:00 to 2025-06-30 00:00
  → Fetched 761 records
Total records fetched: 8761
Data fetch completed successfully!


In [38]:
# Process the data into a pandas DataFrame
if bitcoin_data:
    print("Processing data into DataFrame...")
    
    # Define column names based on Binance kline data format
    columns = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ]
    
    # Create DataFrame
    df = pd.DataFrame(bitcoin_data, columns=columns)
    
    # Convert timestamp columns to datetime
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    
    # Convert price columns to float
    price_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume',
                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
    for col in price_columns:
        df[col] = df[col].astype(float)
    
    # Sort by open_time to ensure chronological order
    df = df.sort_values('open_time').reset_index(drop=True)
    
    # Remove the 'ignore' column as it's not needed
    df = df.drop('ignore', axis=1)
    
    print(f"DataFrame created with {len(df)} rows")
    print(f"Date range: {df['open_time'].min()} to {df['open_time'].max()}")
    
else:
    print("No data to process.")


Processing data into DataFrame...
DataFrame created with 8761 rows
Date range: 2024-06-29 17:00:00 to 2025-06-29 17:00:00


In [39]:
# Display basic information about the data
if 'df' in locals():
    print("=" * 60)
    print("BITCOIN PRICE DATA SUMMARY")
    print("=" * 60)
    
    # Basic info
    print(f"Total records: {len(df):,}")
    print(f"Time period: {df['open_time'].min().strftime('%Y-%m-%d %H:%M')} to {df['open_time'].max().strftime('%Y-%m-%d %H:%M')}")
    print(f"Symbol: {SYMBOL}")
    print(f"Interval: {INTERVAL}")
    print()
    
    # Price statistics
    print("PRICE STATISTICS (USD)")
    print("-" * 30)
    print(f"Highest price: ${df['high'].max():,.2f}")
    print(f"Lowest price: ${df['low'].min():,.2f}")
    print(f"Latest close price: ${df['close'].iloc[-1]:,.2f}")
    print(f"First close price: ${df['close'].iloc[0]:,.2f}")
    
    price_change = df['close'].iloc[-1] - df['close'].iloc[0]
    price_change_pct = (price_change / df['close'].iloc[0]) * 100
    print(f"Total price change: ${price_change:,.2f} ({price_change_pct:+.2f}%)")
    print()
    
    # Volume statistics
    print("VOLUME STATISTICS")
    print("-" * 30)
    print(f"Total volume (BTC): {df['volume'].sum():,.2f}")
    print(f"Average hourly volume (BTC): {df['volume'].mean():.2f}")
    print(f"Max hourly volume (BTC): {df['volume'].max():,.2f}")
    print()
    
    # Display first and last few rows
    print("FIRST 5 RECORDS:")
    print(df[['open_time', 'open', 'high', 'low', 'close', 'volume']].head())
    print()
    print("LAST 5 RECORDS:")
    print(df[['open_time', 'open', 'high', 'low', 'close', 'volume']].tail())


BITCOIN PRICE DATA SUMMARY
Total records: 8,761
Time period: 2024-06-29 17:00 to 2025-06-29 17:00
Symbol: ARBUSDC
Interval: 1h

PRICE STATISTICS (USD)
------------------------------
Highest price: $1.24
Lowest price: $0.24
Latest close price: $0.34
First close price: $0.79
Total price change: $-0.45 (-56.92%)

VOLUME STATISTICS
------------------------------
Total volume (BTC): 3,049,800,941.80
Average hourly volume (BTC): 348111.05
Max hourly volume (BTC): 8,874,874.70

FIRST 5 RECORDS:
            open_time    open    high     low   close    volume
0 2024-06-29 17:00:00  0.7874  0.7879  0.7843  0.7873  235540.4
1 2024-06-29 18:00:00  0.7872  0.7880  0.7848  0.7862   58797.7
2 2024-06-29 19:00:00  0.7858  0.7875  0.7818  0.7869  114933.1
3 2024-06-29 20:00:00  0.7864  0.7880  0.7835  0.7840   75279.3
4 2024-06-29 21:00:00  0.7841  0.7871  0.7817  0.7861   49688.9

LAST 5 RECORDS:
               open_time    open    high     low   close     volume
8756 2025-06-29 13:00:00  0.3110  0.31

In [40]:
# Save data to CSV file
if 'df' in locals():
    filename = f"{SYMBOL}_hourly_data_{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.csv"
    df.to_csv(filename, index=False)
    print(f"Data saved to: {filename}")
    print(f"File size: {len(df):,} rows x {len(df.columns)} columns")
    
    # Optional: Display DataFrame info
    print("\nDataFrame Info:")
    print(df.info())
    
else:
    print("No data available to save.")


Data saved to: ARBUSDC_hourly_data_20240630_20250630.csv
File size: 8,761 rows x 11 columns

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     8761 non-null   datetime64[ns]
 1   open                          8761 non-null   float64       
 2   high                          8761 non-null   float64       
 3   low                           8761 non-null   float64       
 4   close                         8761 non-null   float64       
 5   volume                        8761 non-null   float64       
 6   close_time                    8761 non-null   datetime64[ns]
 7   quote_asset_volume            8761 non-null   float64       
 8   number_of_trades              8761 non-null   int64         
 9   taker_buy_base_asset_volume   8761 non-null   float64